#**Proyecto Final**
**Caso de aplicación guía: Plan de producción en una empresa manufacturera**

Nombres: Andrea Coral, Valentina Gonzales y Diego Ávila

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import scipy.optimize as sco
import warnings
warnings.filterwarnings("ignore")
%pip install --quiet yfinance
import yfinance as yf
from pandas_datareader import data as pdr
import cvxpy as cp
from scipy.stats import norm

La planeación del proceso de producción es uno de los problemas más importantes a los que se enfrentan las industrias manufactureras.

# **Objetivo:**
Optimizar la producción determinando la cantidad óptima de bienes a producir, teniendo en cuenta factores como la demanda, la capacidad de producción y los costos de inventario.

Debido a que el problema es complejo, necesita un modelo de optimización para encontrar la mejor solución.


Supongamos que una empresa manufacturera produce tres tipos de productos: Producto
A, Producto B y Producto C. La empresa tiene tres instalaciones de producción ubicadas
en diferentes ciudades con capacidades instaladas en cada una: 1000, 1200 y 1500,
respectivamente; y tiene que decidir cuánto producir de cada producto en cada instalación
si decide mantenerla abierta y cómo distribuir los tres productos para cubrir una restricción
de la demanda que corresponde a 500, 700 y 800, respectivamente.

# **Caso**

In [73]:
df = pd.DataFrame({' ' : ['Capacidad instalada','Restricción demanda'],
        'Product A' : [1000, 500],
        'Product B' : [1200, 1700],
        'Product C' : [1500, 800]})
df

,,Product A,Product B,Product C
0,Capacidad instalada,1000,1200,1500
1,Restricción demanda,500,1700,800


# **Problema**
Minimizar los costos de producción y
distribución, tomando en cuenta la capacidad de producción de cada instalación, los
costos de transporte y la demanda de cada producto. Por este motivo, debe revisar que
instalación debe cerrar.

Además, la empresa tiene que asumir los siguientes costos de distribución:

In [27]:
df1 = pd.DataFrame({' ' : ['Instalación 1', 'Instalación 2',"Instalación 3"],
        'Punto de demanda 1' : [5,6,9],
        'Punto de demanda 2' : [8,4,5],
        'Punto de demanda 3' : [9,12,10]})
df1

,,Punto de demanda 1,Punto de demanda 2,Punto de demanda 3
0,Instalación 1,5,8,9
1,Instalación 2,6,4,12
2,Instalación 3,9,5,10


# Variables:

X (i,j) = cantidad del producto i producido en la instalación j

d (i) = demanda del producto i

C (i,j) = costo de transportar el producto i desde la instalación j hasta la ubicación de la demanda



In [28]:
# Datos del problema
productos = 3
instalaciones = 3
capacidades = np.array([1000, 1200, 1500])
demandas = np.array([500, 700, 800])
costos = np.array([[5, 8, 9], [6, 4, 12], [9, 5, 10]])

A continuación se realiza una optimización utilizando la librería CVXPY  para resolver el problema de optimización, lo que hace esta librería es minimizar la función objetivo con sus respectivas restricciones, cabe mencionar que tanto la función como las restricciones son lineales.  

In [29]:
# Variables de decisión
x = cp.Variable((productos, instalaciones))

# Función objetivo
costo_total = cp.sum(cp.multiply(costos, x))

# Restricciones
# Restricción de capacidad
for j in range(instalaciones):
    cp.sum(x[:, j]) <= capacidades[j]

# Restricción de demanda
for i in range(productos):
    cp.sum(x[i, :]) == demandas[i]

# Restricción de no negatividad
x >= 0

# Problema de optimización
prob = cp.Problem(cp.Minimize(costo_total), [x >= 0, 
    cp.sum(x[:, 0]) <= capacidades[0], 
    cp.sum(x[:, 1]) <= capacidades[1],
    cp.sum(x[:, 2]) <= capacidades[2],
    cp.sum(x[0, :]) == demandas[0],
    cp.sum(x[1, :]) == demandas[1],
    cp.sum(x[2, :]) == demandas[2]])

# Resolución del problema
prob.solve()

# Resultados

print("Costo total:", prob.value)
print("Cantidad de productos producidos:")
print(x.value)
pd.DataFrame(x.value).round(2)

Costo total: 9900.00002040023
Cantidad de productos producidos:
[[4.99999999e+02 2.92635419e-07 5.84871759e-07]
 [3.00000002e+02 3.99999998e+02 2.31390215e-07]
 [1.97877330e-06 7.99999997e+02 8.47020622e-07]]


,0,1,2
0,500.0,0.0,0.0
1,300.0,400.0,0.0
2,0.0,800.0,0.0


La solución que se obtuvo es la solución discreta y nos muestra que la instalación a cerrar es la número 3. A continuación, se utilizará el método Monte Carlo, el cual genera muestras aleatorias. En este caso el número de muestra es 100.000 para obtener mayor aproximación al resultado óptimo. Además, este método permite simular diferentes situaciones que se puede presentar en el caso de la demanda, es decir, cambios por preferencia de los consumidores o fluctuaciones en el mercado. Por todo lo anterior, el método Monte Carlo complementa al método usado anteriormente al evaluar distintos escenarios mediante la generación de distribucione.

In [8]:
!pip install --quiet pyDOE
from pyDOE import lhs

  Preparing metadata (setup.py) ... done


Se generan las distriuciones para cada restricción de demanda:

In [66]:
ns = 100000
rands_u = lhs(1, ns)
mu = 500
std = 20
rands_d1 = norm(mu, std).ppf(rands_u)

In [67]:
rands_u = lhs(1, ns)
mu = 700
std = 28
rands_d2 = norm(mu, std).ppf(rands_u)

In [68]:
rands_u = lhs(1, ns)
mu = 800
std = 35
rands_d3 = norm(mu, std).ppf(rands_u)

In [69]:
dem = np.concatenate((rands_d1.reshape(-1,1), rands_d2.reshape(-1,1), rands_d3.reshape(-1,1)), axis=1)
dem.shape

(100000, 3)

Por consiguiente, se usa un ciclo for que va a iterar cada resultado de la distribución, es decir, de las 100.000 muestras aleatorias. Luego se aplica una optimización por el método usado anteriormente (CVXPY). 

In [70]:
soluciones = np.zeros(ns)
for k in range(ns):
  demanda = dem[k,:]
  x = cp.Variable((productos, instalaciones))
  costo_total = cp.sum(cp.multiply(costos, x))
  
  for j in range(instalaciones):
    cp.sum(x[:, j]) <= capacidades[j]
  for i in range(productos):
    cp.sum(x[i, :]) == demanda[i]
  x >= 0
  
  prob = cp.Problem(cp.Minimize(costo_total), [x >= 0, 
    cp.sum(x[:, 0]) <= capacidades[0], 
    cp.sum(x[:, 1]) <= capacidades[1],
    cp.sum(x[:, 2]) <= capacidades[2],
    cp.sum(x[0, :]) == demanda[0],
    cp.sum(x[1, :]) == demanda[1],
    cp.sum(x[2, :]) == demanda[2]])
  prob.solve()
  sol = prob.value
  soluciones[k] = sol
  

En consecuencia se obtiene la media de los resultados anteriores y se obtiene como costo toal:

In [72]:
d = soluciones.mean()
print("Costo total:", d)
print("Cantidad de productos producidos:")
print(x.value)
pd.DataFrame(x.value).round(2)

Costo total: 9900.000384021774
Cantidad de productos producidos:
[[4.93306106e+02 4.00784702e-07 7.71822555e-07]
 [3.20823319e+02 3.63211419e+02 3.39947234e-07]
 [2.64258488e-06 8.36788576e+02 1.72618911e-06]]


,0,1,2
0,493.31,0.00,0.0
1,320.82,363.21,0.0
2,0.00,836.79,0.0


**¿Cuál es la decisión optima de la empresa en su plan de producción y de distribución?**
Los resultados demuestran que para minimizar los costos de producción y distribución, la empresa debe cerrar la instalación numero 3. Además, se encontró que la mejor forma de satisfacer las demandas de las 3 instalaciones es que la planta 1 produza 493,31 unidades para el punto 1 y 320,82 unidades para el punto 2. A esto se le suma que la planta 2 produzca 363,21 unidades para el punto 2 y 836,79 para el punto *3*

**¿Cambia la decisión del plan de producción de la empresa si se afectan las variables de
demanda y de costos?**

Si, ya que en el caso de la demanda, si esta aumenta, ellos deben producir más para satisfacer la demanda, es decir, modificar las unidades que se produzcan o, incluso, aumentar la capacidad. Lo mismo ocurre con los costos, ya que si estos aumentan, por ejemplo, deben ajustar su plan de producción con el fin de mantener su rentabilidad. 
